If you need the dependencies, install them

In [30]:
%pip install langchain-community
%pip install langchain

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/817.7 kB ? eta -:--:--
   - -------------------------------------- 30.7/817.7 kB 1.3 MB/s eta 0:00:01
   --------------- ------------------------ 307.2/817.7 kB 3.8 MB/s eta 0:00:01
   ---------------------------------------- 817.7/817.7 kB 7.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


Import Libraries for notebook


In [36]:
import ollama
import json
from collections import deque
from langchain_community.chat_models import ChatOllama
from langchain.schema import HumanMessage, SystemMessage, AIMessage

Basic Example for Ollama

In [2]:
response = ollama.chat(model='llama2', messages=[
    {
        # Who is writing(?)
        'role':'user',
        # The query to the LLM
        'content':'Please write a short story about a data scientist',
    },
])
print(response['message']['content'])


Title: The Data Scientist's Dilemma

Dr. Emma Taylor had always been fascinated by the power of data. As a data scientist at a prominent tech firm, she spent her days poring over numbers and algorithms, trying to make sense of the world around her. But as she delved deeper into her work, she began to realize that there was more to life than just numbers and codes.

Emma's latest project involved analyzing the purchasing habits of a popular retail chain. She had been tasked with identifying patterns and trends in customer behavior, in order to help the company optimize their sales strategies. As she worked tirelessly through the data sets, Emma began to notice something strange.

The customers who were buying the most expensive items were not necessarily the ones with the highest incomes. In fact, some of the biggest spenders were actually low-income individuals, who were willing to splurge on luxury items despite their limited budgets. Emma was intrigued by this phenomenon and decided

Demonstrating that Ollama2 can be used to pull specific details out of a sentence

In [4]:
sys_msg = 'Grab the payer, recipient, amount from any sentence you are given'

response = ollama.chat(model='llama2', messages=[
    {
        'role':'system',
        'content':sys_msg,
    },
    {
        'role':'user',
        'content':'John payed James $50'
    }
])
print(response['message']['content'])


* Payer: John
* Recipient: James
* Amount: $50


Same example but the output can be formatted as JSON, and certain outputs can be massaged to fit a pattern (e.g. say "weekly" instead of "Friday")

In [20]:
sys_msg = """Grab the payer, recipient, amount from any sentence you are given,
    and output it in the following JSON format:
    {{
        "payer":"<name of payer>",
        "recipient":"<name of recipient>",
        "amount":"<amount in USD>",
        "frequency":"<frequency of payment>" # display as "hourly", "daily", "weekly", etc.
    }}
"""

response = ollama.chat(model='llama2', messages=[
    {
        'role':'system',
        'content':sys_msg,
    },
    {
        'role':'user',
        'content':'John payed James $50 every Friday'
    }
])
print(response['message']['content'])

{{
    "payer":"Paul",
    "recipient":"James",
    "amount":"20.00",
    "frequency":"daily"
}}


Testing if Ollama can read multiple lines and output a list of responses, but so far it only seems to output the last one

In [18]:
sys_msg = """Grab the payer, recipient, amount from any sentence you are given,
    and output it in the following JSON format:
    {{
        "payer":"<name of payer>",
        "recipient":"<name of recipient>",
        "amount":"<amount in USD>",
        "frequency":"<frequency of payment>" # display as "hourly", "daily", "weekly", etc.
    }}
"""

inputs = [
    'John pays James $50 every friday',
    'Sam receives $20 from chuck the first of each month',
    'The local pub receives payment from a patron each week'
]

system_mode = {
    'role':'system',
    'content':sys_msg,
}

msgs = [system_mode]

for i in inputs:
    msgs.append(dict({'role':'user', 'content':i}))


response = ollama.chat(model='llama2', messages=msgs)
print(response['message']['content'])

{'model': 'llama2', 'created_at': '2024-04-26T14:28:17.919118Z', 'message': {'role': 'assistant', 'content': '{{\n    "payer":"Chuck",\n    "recipient":"Sam",\n    "amount":"$20",\n    "frequency":"monthly"\n}}'}, 'done': True, 'total_duration': 5958509300, 'load_duration': 1641200, 'prompt_eval_duration': 184933000, 'eval_count': 40, 'eval_duration': 5769440000}


We can use the assistant role to validate expect output against a test set

In [25]:
sys_msg = """Grab the payer, recipient, amount from any sentence you are given,
    and output it in the following JSON format:
    {
        "payer":"<name of payer>",
        "recipient":"<name of recipient>",
        "amount":"<amount in USD>",
        "frequency":"<frequency of payment>" # display as "hourly", "daily", "weekly", etc.
    }
"""

expected_out = {
    "payer":"John James", 
    "recipient":"Paul Tom", 
    "amount":50,
    "pay frequency":"monthly"
    }

response = ollama.chat(model='llama2', messages=[
    {
        'role':'system',
        'content':sys_msg,
    },
    {
        'role':'assistant',
        'content':str(expected_out)
    },
    {
        'role':'user',
        'content':'John James paid Paul Tom $50 every month'
    }
])

print(format)
print(response['message']['content'])

{'payer': 'John James', 'recipient': 'Paul Tom', 'amount': 50, 'pay frequency': 'monthly'}
{{
"payer":"John James",
"recipient":"Paul Tom",
"amount":"50.00",
"pay frequency":"monthly"
}}


Using the LangChain wrapper library to simplify the syntax, and add features

In [38]:
llm = ChatOllama(model="llama2")


sys_msg = """Grab the payer, recipient, amount from any sentence you are given,
    and output it in the following JSON format:
    {
        "payer":"<name of payer>",
        "recipient":"<name of recipient>",
        "amount":"<amount in USD>",
        "frequency":"<frequency of payment>" # display as "hourly", "daily", "weekly", etc.
    }
"""

inputs = [
    'John pays James $50 every friday',
    'Sam receives $20 from chuck the first of each month',
    'The local pub receives payment from a patron each week'
    'Nancy pays Jim, Harry, and Bob $75 each Wednesday'
]

messages = [
    SystemMessage(
        content=sys_msg
    ),
    HumanMessage(
        content=inputs[1]
    )
]

chat_model_response = llm.invoke(messages)
print(chat_model_response.content)

# access the payer information
print(json.loads(chat_model_response.content)['payer'])

{
"payer":"Chuck",
"recipient":"Sam",
"amount":"20.00",
"frequency":"monthly"
}
Chuck


Trying to get Ollama to generate itemized transactions for when a single person pays multiple people in one input string

In [51]:
llm = ChatOllama(model="llama2", format="json") #format="json" for json output


sys_msg = """Grab the payer, recipient, amount from any sentence you are given,
    and output it in the following JSON format:
    {"results":[
        {
        "payer":"<name of payer>",
        "recipient":"<name of recipient>",
        "amount":"<amount in USD>",
        "frequency":"<frequency of payment>" # display as "hourly", "daily", "weekly", etc.
        },
        {
        <ITEM 2>
        },
        {
        <ITEM 3>
        },
    ]}. But separate each individual transaction between two people into a separate item. Print only the JSON output.
"""

inputs = [
    'John pays James $50 every friday',
    'Sam receives $20 from chuck the first of each month',
    'The local pub receives payment from a patron each week',
    'Nancy pays Jim, Harry, and Bob $75 each Wednesday'
]

messages = [
    SystemMessage(
        content=sys_msg
    ),
    HumanMessage(
        content=inputs[3]
    )
]

chat_model_response = llm.invoke(messages)
print(chat_model_response.content)

# access the payer information
json_out = json.loads(chat_model_response.content)
print(json_out['results'][0])

{"results":[
{
"payer":"Nancy",
"recipient":"Jim",
"amount":"$75",
"frequency":"weekly"
},
{
"payer":"Nancy",
"recipient":"Harry",
"amount":"$75",
"frequency":"weekly"
},
{
"payer":"Nancy",
"recipient":"Bob",
"amount":"$75",
"frequency":"weekly"
}
]}
{'payer': 'Nancy', 'recipient': 'Jim', 'amount': '$75', 'frequency': 'weekly'}
